In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n!pip install db-dtypes\n!python3 -m pip install pandas\n!pip install pmdarima\n")

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima

In [ ]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n!pip install db-dtypes\n!python3 -m pip install pandas\n!pip install pmdarima\n!pip install plotly==5.11.0\n")

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima

- DEFINE YOUR CLASSES AND FUNCTIONS 
-----------------------------------
This is not required, but is helpful in keeping your notebook organized. 
You can use the following cell or several cells to define your functions
and classes to keep them separate from your analysis or results code.
In general it useful to define your methods in a separate cell from where
it is run.

In [ ]:
def dataExplore(data):
    '''
    Explore dataframe
    '''
    print("# of observations: ", data.shape[0])
    for col in data.columns:
        if col in ["uu_id", "timeperiod", "week_number", "countyfips", "tract", "tract_name", "date"]:
            print("# of %s: %s" % (col, len(pd.unique(data[col]))))
        else:
            print("Unique value of %s: %s" % (col, pd.unique(data[col])))

In [ ]:
def dataBalanceCheck(data):
    '''
    Check the balance of data frame
    '''
    unbalance_count = 0
    print("# of observations in complete time series: ", len(pd.unique(data["week_number"])))
    for id in pd.unique(data["uu_id"]):
        if len(data[data["uu_id"] == id]) <  len(pd.unique(data["week_number"])):
            print(id, len(data[data["uu_id"] == id]))
            unbalance_count += 1
    print("% of tracts with incomplete time series: ", unbalance_count / len(pd.unique(data["uu_id"]))*100)

In [ ]:
def dataFillNa(data, value):
    """
   fill NA with given value in the dataframe
    """
    for col in data.columns:
        if col in ["uu_id", "timeperiod", "week_number", "countyfips", "tract", "tract_name", "date"]:
            pass
        elif col in ["top_category_employer1", "top_category_employer2", "top_category_employer3"]:
            data[col] = data[col].replace({'N/A':str(value)})
        else:
            data[col] = data[col].fillna(value)
    return(data)

In [ ]:
def dataIdentifyDWM(data):
    '''
    Input: # of week. Output: data for the first day, its month and week order in the month
    '''
    data["date"] = pd.to_datetime(2022 * 1000 + (1+(data["week_number"]-1)*7), format='%Y%j')
    data["month"] = pd.DatetimeIndex(data["date"]).month
    data["weekofmonth"]= pd.to_numeric(data["date"].dt.day/7)
    data['weekofmonth'] = data['weekofmonth'].apply(lambda x: math.ceil(x))
    return(data)

In [ ]:
def MSPE(s1, s2):
    return(sum((s1 - s2)**2)/len(s1))

In [ ]:
def MAPE(s1, s2):
    return(sum(abs(s1 - s2))/len(s1))

In [ ]:
def ARIMA_predict(df_input, cutoff_rate = 0.8, n_period = 15):
    cutoff = int(cutoff_rate * len(df_input))
    if cutoff_rate < 1:
        valid = df_input[cutoff:]
    train = df_input[:cutoff]
    model = auto_arima(train, trace=False, error_action='ignore', suppress_warnings=True)
    model.fit(train)
    forecast = model.predict(n_period)
    return(forecast)

In [ ]:
# Obtain data using BigQuery
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query = """
SELECT
a.*,
b.average_wage
FROM 
(SELECT 
*
FROM `ironhacks-data.ironhacks_competition.unemployment_data`) a
JOIN `ironhacks-data.ironhacks_competition.wage_data` b 
ON a.uu_id=b.uu_id
"""

In [ ]:
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()

In [ ]:
query_pred = """
SELECT * FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
query_job_pred = bigquery_client.query(query_pred)
data_pred_query= query_job_pred.to_dataframe()

In [ ]:
# Explore input data for NA and special values
dataExplore(data)
# dataExplore(data_pred_query)
# data_pred_query.head()

In [ ]:
# Explore input data for NA and special values
# dataExplore(data)
dataExplore(data_pred_query)
# data_pred_query.head()

In [ ]:
# Explore input data for NA and special values
# dataExplore(data)
# dataExplore(data_pred_query)
data_pred_query.head()

In [ ]:
# Explore input data for NA and special values
dataExplore(data)
# dataExplore(data_pred_query)
# data_pred_query.head()

In [ ]:
# Explore input data for NA and special values
# dataExplore(data)
# dataExplore(data_pred_query)
# data_pred_query.head()
data.head()

In [ ]:
# Explore input data for NA and special values
dataExplore(data)
# dataExplore(data_pred_query)
# data_pred_query.head()
data.head()

In [ ]:
# Explore input data for NA and special values
# dataExplore(data)
# dataExplore(data_pred_query)
# data_pred_query.head()
data.head()

In [ ]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n!pip install db-dtypes\n!python3 -m pip install pandas\n!pip install pmdarima\n!pip install plotly==5.11.0\n!pip install hts_reconciliation\n")

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima
import reconciliation_hts

In [ ]:
%%capture

In [ ]:
#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED
#------------------------------------------
# This is normally not required. The hub environment comes preinstaled with 
# many packages that you can already use without setup. In case there is some
# other library you would like to use that isn't on the list you run this command
# once to install them.  If it is already installed this command has no effect.
get_ipython().system('pip install db-dtypes')
get_ipython().system('python3 -m pip install pandas')
get_ipython().system('pip install pmdarima')
get_ipython().system('pip install plotly==5.11.0')
get_ipython().system('pip install hts_reconciliation')

In [ ]:
%%capture

In [ ]:
#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED
#------------------------------------------
# This is normally not required. The hub environment comes preinstaled with 
# many packages that you can already use without setup. In case there is some
# other library you would like to use that isn't on the list you run this command
# once to install them.  If it is already installed this command has no effect.
get_ipython().system('pip install db-dtypes')
get_ipython().system('python3 -m pip install pandas')
get_ipython().system('pip install pmdarima')
get_ipython().system('pip install plotly==5.11.0')
get_ipython().system('pip install scikit-hts[auto-arima]')

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima
import hts
from hts.hierarchy import HierarchyTree
from hts.model import AutoArimaModel
from hts import HTSRegressor

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima
import hts

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima
from hts.hierarchy import HierarchyTree
from hts.model import AutoArimaModel
from hts import HTSRegressor

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima
from collections.abc import Iterable
import hts
from hts.hierarchy import HierarchyTree
from hts.model import AutoArimaModel
from hts import HTSRegressor

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima
from collections.abc import Iterable

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math
import plotly.express as px
from pmdarima.arima import auto_arima
from collections.abc import Iterable
import hts